In [129]:
from PIL import Image
from glob import glob
import cv2
from cv2 import aruco
import numpy as np
from sympy import *
import math
from PIL import ExifTags
from xml.etree import ElementTree as ET
import utm
import math

In [130]:
Xsensor = 0.00617
Ysensor = 0.00455
Xpixel = 4000.
Ypixel = 3000.

In [131]:
bilder = glob('../bildverband2/*.JPG')

bilderDaten = []

for bild in bilder:
    bildDaten = []
    img = Image.open(bild)
    exif = img._getexif()
    for tag, value in exif.items():
        if 'GPSInfo' == ExifTags.TAGS.get(tag, tag):
            n = float(value[2][0])+float(value[2][1]) / \
                60.+float(value[2][2])/3600.
            e = float(value[4][0])+float(value[4][1]) / \
                60.+float(value[4][2])/3600.
            h = float(value[6])
            u = utm.from_latlon(n,e, 32)
            bildDaten.append(u[0])
            bildDaten.append(u[1])
            bildDaten.append(h)
    with open(bild, "rb") as f:
        s = str(f.read())

    start = s.find('<x:xmpmeta')
    end = s.find('</x:xmpmeta')
    xmp = s[start:end+12].replace("\\n", "\n")
    tree = ET.XML(xmp)

    bildDaten.append(float(tree[0][0].attrib['{http://www.dji.com/drone-dji/1.0/}FlightRollDegree'])/180*math.pi)
   
    bildDaten.append(float(
        tree[0][0].attrib['{http://www.dji.com/drone-dji/1.0/}FlightPitchDegree'])/180*math.pi)
    bildDaten.append(float(
        tree[0][0].attrib['{http://www.dji.com/drone-dji/1.0/}FlightYawDegree'])/180*math.pi)
    #bildDaten.append(float(tree[0][0].attrib['{http://www.dji.com/drone-dji/1.0/}RelativeAltitude'])/180*math.pi)
    bilderDaten.append(bildDaten)
bilderDaten = np.array(bilderDaten)


## Marker identifizieren

In [132]:

LUT_IN = [0, 158, 216, 255]
LUT_OUT = [0, 22, 80, 176]
lut = np.interp(np.arange(0, 256),
                LUT_IN, LUT_OUT).astype(np.uint8)

aruco_dict = aruco.Dictionary_create(32, 3)
beobachtungen = []

markers = set()
for bildnr in range(len(bilder)):
    bild = bilder[bildnr]
    cv_img = cv2.imread(bild)
    tmp = cv2.LUT(cv_img, lut)
    gray = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
    
    parameter = aruco.DetectorParameters.create()
    parameter.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX;
    corners, ids, _ = aruco.detectMarkers(gray, aruco_dict, parameters=parameter)

    for nr in range(len(ids)):
        for cid in range(len(corners[nr][0])):
            markers.add(ids[nr][0]*10 + cid)
            beobachtungen.append(
                (bildnr, ids[nr][0]*10 + cid, 0, (corners[nr][0][cid][0]/Xpixel-0.5)*Xsensor))
            beobachtungen.append(
                    (bildnr, ids[nr][0]*10 + cid, 1, (corners[nr][0][cid][1]/Ypixel-0.5)*Ysensor))

    #marked = aruco.drawDetectedMarkers(cv_img, corners, ids)
    #cv2.imshow('image', cv2.resize(marked, (800,600)))
    # cv2.waitKey(0)
beobachtungen


[0.0014138036816406247,
 0.00021894986572265634,
 0.0015216374615478518,
 0.00019819789632161456,
 0.0015057982159423825,
 0.0003215237060546875,
 0.0013986031274414062,
 0.00034324062500000024,
 -0.002087404224090576,
 -0.0002498401000976563,
 -0.0018763151138305666,
 -0.0002919959167480469,
 -0.0018259843669128418,
 -0.00010925406087239584,
 -0.002032868413543701,
 -6.433447469075521e-05,
 0.0011840536541748048,
 -0.00021679372558593756,
 0.0012470197393798827,
 -0.00018852655436197915,
 0.0012355425000000002,
 -0.0001046500000000001,
 0.0011777744439697264,
 -0.00013000380859374994,
 -0.001151144100341797,
 -0.0006443376525878906,
 -0.0011615024999999998,
 -0.0008629833333333333,
 -0.00040259249999999986,
 -0.0009191000000000001,
 -0.0004920574999999999,
 -0.0007128333333333334,
 0.002626414674682617,
 0.0005055846842447915,
 0.0027998160021972655,
 0.0005317155253092449,
 0.002764469931030273,
 0.0006678415425618493,
 0.0025934331927490234,
 0.0006404495279947918,
 -0.0030783036734

In [143]:

l = [i[len(i)-1] for i in beobachtungen]
l


[0.0014138036816406247,
 0.00021894986572265634,
 0.0015216374615478518,
 0.00019819789632161456,
 0.0015057982159423825,
 0.0003215237060546875,
 0.0013986031274414062,
 0.00034324062500000024,
 -0.002087404224090576,
 -0.0002498401000976563,
 -0.0018763151138305666,
 -0.0002919959167480469,
 -0.0018259843669128418,
 -0.00010925406087239584,
 -0.002032868413543701,
 -6.433447469075521e-05,
 0.0011840536541748048,
 -0.00021679372558593756,
 0.0012470197393798827,
 -0.00018852655436197915,
 0.0012355425000000002,
 -0.0001046500000000001,
 0.0011777744439697264,
 -0.00013000380859374994,
 -0.001151144100341797,
 -0.0006443376525878906,
 -0.0011615024999999998,
 -0.0008629833333333333,
 -0.00040259249999999986,
 -0.0009191000000000001,
 -0.0004920574999999999,
 -0.0007128333333333334,
 0.002626414674682617,
 0.0005055846842447915,
 0.0027998160021972655,
 0.0005317155253092449,
 0.002764469931030273,
 0.0006678415425618493,
 0.0025934331927490234,
 0.0006404495279947918,
 -0.0030783036734

In [144]:
sym = []

innereOrientierung = [
    Symbol('c'),
    Symbol('deltax'),
    Symbol('deltay')
]

sym.extend(innereOrientierung)

bildOrientierung = []
for bild in range(len(bilder)):
    bildO = []
    bildO.append(Symbol('bild'+str(bild)+'_x'))
    bildO.append(Symbol('bild'+str(bild)+'_y'))
    bildO.append(Symbol('bild'+str(bild)+'_z'))
    bildO.append(Symbol('bild'+str(bild)+'_kappa'))
    bildO.append(Symbol('bild'+str(bild)+'_omega'))
    bildO.append(Symbol('bild'+str(bild)+'_phi'))
    bildOrientierung.append(bildO)
    sym.extend(bildO)

neupunkte = []
for marker in list(markers):
    neup = []
    neup.append(Symbol('marker'+str(marker)+'x'))
    neup.append(Symbol('marker'+str(marker)+'y'))
    neup.append(Symbol('marker'+str(marker)+'z'))
    neupunkte.append(neup)
    sym.extend(neup)

print(sym)


[c, deltax, deltay, bild0_x, bild0_y, bild0_z, bild0_kappa, bild0_omega, bild0_phi, bild1_x, bild1_y, bild1_z, bild1_kappa, bild1_omega, bild1_phi, bild2_x, bild2_y, bild2_z, bild2_kappa, bild2_omega, bild2_phi, bild3_x, bild3_y, bild3_z, bild3_kappa, bild3_omega, bild3_phi, bild4_x, bild4_y, bild4_z, bild4_kappa, bild4_omega, bild4_phi, bild5_x, bild5_y, bild5_z, bild5_kappa, bild5_omega, bild5_phi, marker0x, marker0y, marker0z, marker1x, marker1y, marker1z, marker2x, marker2y, marker2z, marker3x, marker3y, marker3z, marker260x, marker260y, marker260z, marker261x, marker261y, marker261z, marker262x, marker262y, marker262z, marker263x, marker263y, marker263z, marker10x, marker10y, marker10z, marker11x, marker11y, marker11z, marker12x, marker12y, marker12z, marker13x, marker13y, marker13z, marker140x, marker140y, marker140z, marker141x, marker141y, marker141z, marker142x, marker142y, marker142z, marker143x, marker143y, marker143z, marker30x, marker30y, marker30z, marker31x, marker31y, m

In [145]:
from random import random

x0 = []

bilderDatenFlat = bilderDaten.flatten()

for nr, __ in enumerate(sym):
    if nr == 0:  # Brennweite
        x0.append(0.0045)
    elif nr == 1:  # Fokuspunkt
        x0.append(0)
    elif nr == 2:  # Fokuspunkt
        x0.append(0)
    elif nr > 2 and nr < len(bilderDatenFlat)+3:
        x0.append(bilderDatenFlat[nr-3])
    elif (nr-3) % 3 == 0:
        x0.append(bilderDatenFlat[0]+(random()-5)*10)
    elif (nr-3) % 3 == 1:
        x0.append(bilderDatenFlat[1]+(random()-5)*10)
    elif (nr-3) % 3 == 2:
        x0.append(bilderDatenFlat[2]+(random()-5)*10)
    else:
        x0.append(0)


siehe Seite 281 Nahbereichsphotogrammetrie

Nahbereichsphotogrammetrie S. 274

In [135]:
formeln = []


def r11(kappa, omega, phi):
    return cos(phi)*cos(kappa)-sin(phi)*sin(omega)*sin(kappa)


def r21(kappa, omega, phi):
    return sin(phi)*cos(kappa)+cos(phi)*sin(omega)*sin(kappa)


def r31(kappa, omega, phi):
    return -cos(omega)*sin(kappa)


def r12(kappa, omega, phi):
    return -sin(phi)*cos(omega)


def r22(kappa, omega, phi):
    return cos(phi)*cos(omega)


def r32(kappa, omega, phi):
    return sin(omega)


def r13(kappa, omega, phi):
    return cos(phi)*sin(kappa)+sin(phi)*sin(omega)*cos(kappa)


def r23(kappa, omega, phi):
    return sin(phi)*sin(kappa)-cos(phi)*sin(omega)*cos(kappa)


def r33(kappa, omega, phi):
    return cos(omega)*cos(kappa)


def kx(x, x0, y, y0, z, z0, kappa, omega, phi):
    return r11(kappa, omega, phi)*(x-x0)+r21(kappa, omega, phi)*(y-y0)+r31(kappa, omega, phi)*(z-z0)


def ky(x, x0, y, y0, z, z0, kappa, omega, phi):
    return r12(kappa, omega, phi)*(x-x0)+r22(kappa, omega, phi)*(y-y0)+r32(kappa, omega, phi)*(z-z0)


def nenner(x, x0, y, y0, z, z0, kappa, omega, phi):
    return r13(kappa, omega, phi)*(x-x0)+r23(kappa, omega, phi)*(y-y0)+r33(kappa, omega, phi)*(z-z0)


def x_kolinear(c, x, x0, y, y0, z, z0, kappa, omega, phi, deltax):
    return x0+c*kx(x, x0, y, y0, z, z0, kappa, omega, phi)/nenner(x, x0, y, y0, z, z0, kappa, omega, phi) + deltax


def y_kolinear(c, x, x0, y, y0, z, z0, kappa, omega, phi, deltay):
    return y0+c*ky(x, x0, y, y0, z, z0, kappa, omega, phi)/nenner(x, x0, y, y0, z, z0, kappa, omega, phi) + deltay


def kolinear(xy, c, x0, y0, z0, kappa, omega, phi, x, y, z, deltax, deltay):
    if xy == 0:
        return x_kolinear(c, x, x0, y, y0, z, z0, kappa, omega, phi, deltax)
    return y_kolinear(c, x, x0, y, y0, z, z0, kappa, omega, phi, deltay)

In [146]:
formeln = []
for bnr in range(len(beobachtungen)):
    bildnr, punkt, xy, koord = beobachtungen[bnr]
    ori = bildOrientierung[bildnr]
    bo = bildOrientierung[bildnr]
    neup = neupunkte[list(markers).index(punkt)]
    formeln.append(kolinear(
        xy, innereOrientierung[0], bo[0], bo[1], bo[2], bo[3], bo[4], bo[5], neup[0], neup[1], neup[2], innereOrientierung[1],innereOrientierung[2]))


formeln.append(innereOrientierung[0])
l.append(x0[0])

for bild in range(len(bildOrientierung)):
    for bildP in range(3):
        formeln.append(bildOrientierung[bild][bildP])
        l.append(bilderDaten[bild][bildP])


In [147]:
formeln

[bild0_x + c*((-bild0_x + marker0x)*(-sin(bild0_kappa)*sin(bild0_omega)*sin(bild0_phi) + cos(bild0_kappa)*cos(bild0_phi)) + (-bild0_y + marker0y)*(sin(bild0_kappa)*sin(bild0_omega)*cos(bild0_phi) + sin(bild0_phi)*cos(bild0_kappa)) - (-bild0_z + marker0z)*sin(bild0_kappa)*cos(bild0_omega))/((-bild0_x + marker0x)*(sin(bild0_kappa)*cos(bild0_phi) + sin(bild0_omega)*sin(bild0_phi)*cos(bild0_kappa)) + (-bild0_y + marker0y)*(sin(bild0_kappa)*sin(bild0_phi) - sin(bild0_omega)*cos(bild0_kappa)*cos(bild0_phi)) + (-bild0_z + marker0z)*cos(bild0_kappa)*cos(bild0_omega)) + deltax,
 bild0_y + c*(-(-bild0_x + marker0x)*sin(bild0_phi)*cos(bild0_omega) + (-bild0_y + marker0y)*cos(bild0_omega)*cos(bild0_phi) + (-bild0_z + marker0z)*sin(bild0_omega))/((-bild0_x + marker0x)*(sin(bild0_kappa)*cos(bild0_phi) + sin(bild0_omega)*sin(bild0_phi)*cos(bild0_kappa)) + (-bild0_y + marker0y)*(sin(bild0_kappa)*sin(bild0_phi) - sin(bild0_omega)*cos(bild0_kappa)*cos(bild0_phi)) + (-bild0_z + marker0z)*cos(bild0_kappa)

In [148]:
A_vorlage = []
for f in range(len(formeln)):
    zeile = []
    for s in range(len(sym)):
        formel = formeln[f].diff(sym[s])
        zeile.append(formel)
    A_vorlage.append(zeile)


In [149]:

P = np.eye(len(l))
print('P', P)


P [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


## Ab hier Schleife...

In [150]:
werte = {}

for nr in range(len(sym)):
    werte[sym[nr]] = x0[nr]

#print(werte)

A = np.empty((len(formeln), len(sym)))
for f in range(len(formeln)):
    for s in range(len(sym)):
        A[f][s] = A_vorlage[f][s].evalf(subs=werte)


l0 = np.empty(len(l))
for i in range(len(l0)):
    l0[i] = formeln[i].evalf(subs=werte)

dl = l - l0

N = np.transpose(A)@P@A
h = np.transpose(A)@P@dl
Qx = np.linalg.inv(N)
dx = Qx@h

x0 = x0 + dx

#print('x0', x0)

dx.sum()


-8.693628012910694e+27

In [151]:
dx

array([ 9.80118372e+07,  9.47535927e+08, -5.36510938e+08, -1.46398381e+08,
        1.30453392e+08,  9.09958733e+04, -9.82697829e+11, -3.49317434e+12,
       -3.39365141e+12, -1.00753374e+08,  3.87023069e+08,  4.46959226e+03,
       -8.47452769e+10, -1.16628164e+11, -2.16571837e+11,  4.73162413e+08,
        2.44015761e+08,  2.45373915e+04,  1.76322564e+12,  1.46598473e+12,
        1.56142268e+12,  7.95439052e+08,  2.68890353e+08,  2.38005016e+04,
        2.71321337e+12,  2.34487924e+12,  2.33943946e+12,  1.60122165e+08,
        2.62493610e+07,  1.36241378e+04,  7.66670383e+11,  1.60737764e+12,
        1.15228613e+12, -1.01139011e+09, -1.14190326e+09, -1.72457257e+05,
        3.74920939e+12,  5.06638789e+12,  3.83859678e+12, -1.64810239e+15,
       -1.58268744e+15, -1.45895619e+15, -3.56176330e+14, -3.79520045e+14,
       -3.90459003e+14,  1.27458273e+15,  9.55543547e+14,  9.64954904e+14,
       -9.09896914e+14, -8.89813430e+14, -8.76023270e+14,  1.17920581e+26,
        1.17091229e+26,  